In [1]:
from pyspark.sql import SQLContext
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler
from notebooks import utils
%matplotlib inline
import pyspark
from pyspark.sql.functions import col,lit
from pyspark.sql.types import StructType,StructField,StringType,IntegerType
import time

In [2]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
df = sqlContext.read.load('file:///home/cloudera/Downloads/PUBG.csv', 
                          format='com.databricks.spark.csv', 
                          header='true',inferSchema='true')

In [5]:
df.show(5)

+--------------+--------------+--------------+-------+------+-----------+-----+-------------+-----+---------+----------+-----+-----------+-----------+-------------+---------+--------+---------+----------+-------+------------+---------+------------+---------+---------------+------------+---------------+---------+------------+
|            Id|       groupId|       matchId|assists|boosts|damageDealt|DBNOs|headshotKills|heals|killPlace|killPoints|kills|killStreaks|longestKill|matchDuration|matchType|maxPlace|numGroups|rankPoints|revives|rideDistance|roadKills|swimDistance|teamKills|vehicleDestroys|walkDistance|weaponsAcquired|winPoints|winPlacePerc|
+--------------+--------------+--------------+-------+------+-----------+-----+-------------+-----+---------+----------+-----+-----------+-----------+-------------+---------+--------+---------+----------+-------+------------+---------+------------+---------+---------------+------------+---------------+---------+------------+
|7f96b2f878858a|4d4

In [7]:
cols=df.columns
df.printSchema()

root
 |-- Id: string (nullable = true)
 |-- groupId: string (nullable = true)
 |-- matchId: string (nullable = true)
 |-- assists: integer (nullable = true)
 |-- boosts: integer (nullable = true)
 |-- damageDealt: double (nullable = true)
 |-- DBNOs: integer (nullable = true)
 |-- headshotKills: integer (nullable = true)
 |-- heals: integer (nullable = true)
 |-- killPlace: integer (nullable = true)
 |-- killPoints: integer (nullable = true)
 |-- kills: integer (nullable = true)
 |-- killStreaks: integer (nullable = true)
 |-- longestKill: double (nullable = true)
 |-- matchDuration: integer (nullable = true)
 |-- matchType: string (nullable = true)
 |-- maxPlace: integer (nullable = true)
 |-- numGroups: integer (nullable = true)
 |-- rankPoints: integer (nullable = true)
 |-- revives: integer (nullable = true)
 |-- rideDistance: double (nullable = true)
 |-- roadKills: integer (nullable = true)
 |-- swimDistance: double (nullable = true)
 |-- teamKills: integer (nullable = true)
 |--

In [9]:
import pandas as pd
pd.DataFrame(df.take(5),columns=df.columns).transpose()

,0,1,2,3,4
Id,7f96b2f878858a,eef90569b9d03c,1eaf90ac73de72,4616d365dd2853,315c96c26c9aac
groupId,4d4b580de459be,684d5656442f9e,6a4a42c3245a74,a930a9c79cd721,de04010b3458dd
matchId,a10357fd1a4a91,aeb375fc57110c,110163d8bb94ae,f1f1f4ef412d7e,6dc8ff871e21e6
assists,0,0,1,0,0
boosts,0,0,0,0,0
damageDealt,0,91.47,68,32.9,100
DBNOs,0,0,0,0,0
headshotKills,0,0,0,0,0
heals,0,0,0,0,0
killPlace,60,57,47,75,45


In [10]:
df.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
assists,4446966,0.23381492010507837,0.5885730210230652,0,22
boosts,4446966,1.1069077209045448,1.7157934844642069,0,33
damageDealt,4446966,130.7171378984272,170.78060146503475,0.0,6616.0
DBNOs,4446966,0.6578755043326169,1.1457425960775853,0,53
headshotKills,4446966,0.22681958890623405,0.602155208079661,0,64
heals,4446966,1.370147196987789,2.6799819292019107,0,80
killPlace,4446966,47.59935020865912,27.462933930580867,1,101
killPoints,4446966,505.00604209701623,627.5048257251201,0,2170
kills,4446966,0.9247833241810259,1.5584444780020816,0,72


In [12]:
#The column of the dataframe
len(df.columns)

29

In [13]:
#The total count of the dataframe
df.count()

4446966

In [3]:
#drop na
df=df.na.drop(subset=['winPlacePerc'])

In [15]:
df.count()

4446965

In [5]:
#check the number of people in a match
playersJoined=df.groupby(df.matchId).count()

In [7]:
playersJoined.printSchema()

root
 |-- matchId: string (nullable = true)
 |-- count: long (nullable = false)



In [60]:
playersJoined.sort('count',ascending=False).show()

+--------------+-----+
|       matchId|count|
+--------------+-----+
|a7367cf85d0db9|  100|
|21f34ed30102cd|  100|
|e90703b941c398|  100|
|fd980efc1b5c3d|  100|
|f0c1de80afa8e7|  100|
|7d005fbc3e7cfa|  100|
|a39df882642c4d|  100|
|06b3f2c58518f4|  100|
|1a31f480716898|  100|
|cd5651b2decf05|  100|
|4020b45808feef|  100|
|045043f888f6a7|  100|
|f44c7500c941ce|  100|
|9a5219783267c3|  100|
|e23056639b3d7e|  100|
|7f49b2f144bf03|  100|
|654736efda21d9|  100|
|5a67a3d9a84cfb|  100|
|997a1a90fe3453|  100|
|c4f47bf73eb13e|  100|
+--------------+-----+
only showing top 20 rows



In [61]:
playersJoined.sort('count',ascending=True).show()

+--------------+-----+
|       matchId|count|
+--------------+-----+
|e5a77433bc436f|    2|
|8aa5066c4b6a79|    5|
|9f2b49358564ea|    6|
|39d2800dad8ae6|    6|
|f491e7a7e2ff47|    7|
|6fd4b1adb6da3c|    7|
|da121c891ea777|    7|
|8799301e853202|    7|
|3270384443bd32|    7|
|66ad63285e8952|    7|
|17536f834bb256|    7|
|1f251de6c42568|    7|
|23c3819550f61f|    7|
|0ac026a48eb471|    7|
|7c785a64ec89df|    7|
|bc10cc08f1f56a|    7|
|998a86ce7ec062|    7|
|8772c5274a0630|    8|
|a46b1598692634|    8|
|93513672bce7d5|    8|
+--------------+-----+
only showing top 20 rows



In [6]:
#Add playersJoined in the dataframe
cond=[df.matchId==playersJoined.matchId]
df_join=df.join(playersJoined,on=cond,how="full").drop(playersJoined.matchId)

In [7]:
df2=df_join

In [8]:
df2=df2.na.drop(subset=['winPlacePerc'])
#drop the unuseful column
df2=df2.drop("count")

In [9]:
#change the type of playersjoined into 'int'
df2=df2.withColumn("playersJoin",df2["count"].cast("Int"))

In [10]:
#Normalize some factors according to the number of persons in a match

In [11]:
df2=df2.withColumn("killsNorm",df2.kills*((100-df2.playersJoin)/100+1))
df2=df2.withColumn("damageDealtNorm",df2.damageDealt*((100-df2.playersJoin)/100+1))
df2=df2.withColumn("maxPlaceNorm",df2.maxPlace*((100-df2.playersJoin)/100+1))
df2=df2.withColumn("matchDurationNorm",df2.matchDuration*((100-df2.playersJoin)/100+1))

In [94]:
#show the normalization result
df2.select("Id","kills","killsNorm","damageDealt","damageDealtNorm","maxPlace"
           ,"maxPlaceNorm","matchDuration","matchDurationNorm").show(20)

+--------------+-----+------------------+-----------+------------------+--------+------------+-------------+-----------------+
|            Id|kills|         killsNorm|damageDealt|   damageDealtNorm|maxPlace|maxPlaceNorm|matchDuration|matchDurationNorm|
+--------------+-----+------------------+-----------+------------------+--------+------------+-------------+-----------------+
|ae726f90d45955|    0|               0.0|        0.0|               0.0|      26|        27.3|         1265|          1328.25|
|5045feb853c423|    0|               0.0|        0.0|               0.0|      26|        27.3|         1265|          1328.25|
|619d35699d01d4|    1|              1.05|      91.26| 95.82300000000001|      26|        27.3|         1265|          1328.25|
|aefe8868836fc0|    0|               0.0|        0.0|               0.0|      26|        27.3|         1265|          1328.25|
|fc14e9980f86ff|    0|               0.0|        0.0|               0.0|      26|        27.3|         1265|   

In [22]:
#Create new feature
df2=df2.withColumn("healsandboosts",df2.heals+df2.boosts)
#Create new feature to calculate the total moving distance of the players
df2=df2.withColumn("totalDistance",df2.rideDistance+df2.walkDistance+df2.swimDistance)

In [97]:
df2.select("healsandboosts").show(10)

+--------------+
|healsandboosts|
+--------------+
|             0|
|             0|
|             3|
|             1|
|             0|
|             3|
|             0|
|             1|
|             0|
|             8|
+--------------+
only showing top 10 rows



In [ ]:
#delete the outliers

#Remove players with kills but no moves
cond1=(df2.totalDistance==0)&(df2.kills>0)
df2=df2.filter(~cond1)

#delete abnormal roadkills players
cond2=(df2.roadKills>10)
df2=df2.filter(~cond2)

#delete abnormal kills player
cond3=(df2.kills>30)
df2=df2.filter(~cond3)

#delete abnormal headshot
df2=df2.withColumn("headshotrate",df2.headshotKills/df2.kills)
df2=df2.fillna(0,subset='headshotrate')
cond4=(df2.headshotrate==1)&(df2.kills>9)
df2=df2.filter(~cond4)

#delete the abnormal data of longest kill distance
cond5=(df2.longestKill>1000)
df2=df2.filter(~cond5)

#delete the abnormal walkdistance
cond6=(df2.walkDistance>10000)
df2=df2.filter(~cond6)

#delete the abnormal ridedistance
cond7=(df2.rideDistance>20000)
df2=df2.filter(~cond7)

#delete the abnormal swimdistance
cond8=(df2.swimDistance>2000)
df2=df2.filter(~cond8)

#delete the abnormal weaponsAcquired
cond9=(df2.weaponsAcquired>=80)
df2=df2.filter(~cond9)

#delete the abnormal heals
cond10=(df2.heals>=40)
df2=df2.filter(~cond10)

#delete the player joined the match less than 30 people
cond11=(df2.playersJoin<=30)
df2=df2.filter(~cond11)

In [ ]:
#categorical features processing

In [ ]:
#one hot processing of matchType

In [55]:
from pyspark.ml.feature import OneHotEncoder,StringIndexer,VectorAssembler

In [56]:
matchType_indexer=StringIndexer(inputCol="matchType",outputCol="matchType_index").fit(df2)
df2=matchType_indexer.transform(df2)
matchType_encoder=OneHotEncoder(inputCol="matchType_index",outputCol="matchType_vec")
df2=matchType_encoder.transform(df2)

In [269]:
df2.select("matchType","matchType_index","matchType_vec").show(10)

+---------+---------------+--------------+
|matchType|matchType_index| matchType_vec|
+---------+---------------+--------------+
|    squad|            2.0|(15,[2],[1.0])|
|    squad|            2.0|(15,[2],[1.0])|
|    squad|            2.0|(15,[2],[1.0])|
|    squad|            2.0|(15,[2],[1.0])|
|    squad|            2.0|(15,[2],[1.0])|
|    squad|            2.0|(15,[2],[1.0])|
|    squad|            2.0|(15,[2],[1.0])|
|    squad|            2.0|(15,[2],[1.0])|
|    squad|            2.0|(15,[2],[1.0])|
|    squad|            2.0|(15,[2],[1.0])|
+---------+---------------+--------------+
only showing top 10 rows



In [ ]:
#transform matchId and groupId into indexer

In [60]:
matchId_indexer=StringIndexer(inputCol="matchId",outputCol="matchId_index").fit(df2)
df2=matchId_indexer.transform(df2)
groupId_indexer=StringIndexer(inputCol="groupId",outputCol="groupId_index").fit(df2)
df2=groupId_indexer.transform(df2)

In [279]:
df2.select('matchId','groupId','matchId_index','groupId_index').show(10)

+--------------+--------------+-------------+-------------+
|       matchId|       groupId|matchId_index|groupId_index|
+--------------+--------------+-------------+-------------+
|d175baf70ed648|ca2b3b4a3767a5|      23439.0|      54681.0|
|d175baf70ed648|7dbbbfc823ee84|      23439.0|     291181.0|
|d175baf70ed648|04466adcaf854a|      23439.0|     326142.0|
|d175baf70ed648|d6f5359f685490|      23439.0|     109616.0|
|d175baf70ed648|bbfb5dbbe3388e|      23439.0|      92853.0|
|d175baf70ed648|62cc685c916e51|      23439.0|     187937.0|
|d175baf70ed648|59a0a939ac4942|      23439.0|     291474.0|
|d175baf70ed648|04466adcaf854a|      23439.0|     326142.0|
|d175baf70ed648|62cc685c916e51|      23439.0|     187937.0|
|d175baf70ed648|78d57101399c32|      23439.0|     496997.0|
+--------------+--------------+-------------+-------------+
only showing top 10 rows



In [ ]:
#feature selection

In [333]:
corr={}
for col in df2.columns:
    if col!="winPlacePerc":
        corr[col]=abs(df2.stat.corr(col,'winPlacePerc'))

In [336]:
corr

{'DBNOs': 0.2807155334978206,
 'assists': 0.30049954868817236,
 'boosts': 0.6344589042654931,
 'damageDealtNorm': 0.42746789059909407,
 'groupId_index': 0.1501161949177794,
 'headshotKills': 0.28009697957913127,
 'headshotrate': 0.18101963817689934,
 'heals': 0.42924556353139326,
 'healsandboosts': 0.5771262421553316,
 'killPlace': 0.7192503342850658,
 'killPoints': 0.012949205434866357,
 'killStreaks': 0.3778581700850878,
 'killsNorm': 0.4082605988465656,
 'longestKill': 0.4107286425427856,
 'matchDurationNorm': 0.012123488527207224,
 'matchId_index': 0.0032119562184513978,
 'matchType_index': 0.016637808300621344,
 'maxPlaceNorm': 0.03692710965102434,
 'numGroups': 0.03981353054252559,
 'playersJoin': 0.01799710855244391,
 'rankPoints': 0.01350650744556773,
 'revives': 0.24101341061228487,
 'rideDistance': 0.34406917491611294,
 'roadKills': 0.03492786223878848,
 'swimDistance': 0.150852602635124,
 'teamKills': 0.01598572059338612,
 'totalDistance': 0.6784808196306544,
 'vehicleDestro

In [ ]:
#select features whose correlations are larger than 0.4

In [338]:
feature_corr={}
for key in corr.keys():
    if corr[key]>=0.4:
        feature_corr[key]=corr[key]
        
feature_corr

{'boosts': 0.6344589042654931,
 'damageDealtNorm': 0.42746789059909407,
 'heals': 0.42924556353139326,
 'healsandboosts': 0.5771262421553316,
 'killPlace': 0.7192503342850658,
 'killsNorm': 0.4082605988465656,
 'longestKill': 0.4107286425427856,
 'totalDistance': 0.6784808196306544,
 'walkDistance': 0.8122891571127386,
 'weaponsAcquired': 0.5919674800335318}

In [ ]:
# min max scaler normalization

In [70]:
df_norm=df2.select('healsandboosts','damageDealtNorm','killsNorm','killPlace','totalDistance',
                   'killStreaks','longestKill','walkDistance','weaponsAcquired','winPlacePerc')

In [71]:
from pyspark.sql.functions import min,max

In [72]:
df_norm=df_norm.withColumn("walkDistancenorm",(df_norm.walkDistance-0)/(9990-0))
df_norm=df_norm.withColumn("weaponsAcquirednorm",(df_norm.weaponsAcquired-0)/(76-0))
df_norm=df_norm.withColumn("longestKillnorm",(df_norm.longestKill-0)/(1000-0))
df_norm=df_norm.withColumn("killStreaksnorm",(df_norm.killStreaks-0)/(16-0))
df_norm=df_norm.withColumn("totalDistancenorm",(df_norm.totalDistance-0)/(22521-0))
df_norm=df_norm.withColumn("killPlacenorm",(df_norm.killPlace-1)/(101-1))
df_norm=df_norm.withColumn("killsNormnorm",(df_norm.killsNorm-0)/(50.4-0))
df_norm=df_norm.withColumn("damageDealNormnorm",(df_norm.damageDealtNorm-0)/(5112.24-0))
df_norm=df_norm.withColumn("healsandboostsnorm",(df_norm.healsandboosts-0)/(50-0))

In [100]:
df_norm.select('healsandboosts','healsandboostsnorm','killPlace','killPlacenorm').show()

+--------------+------------------+---------+-------------+
|healsandboosts|healsandboostsnorm|killPlace|killPlacenorm|
+--------------+------------------+---------+-------------+
|             0|               0.0|       80|         0.79|
|             0|               0.0|       44|         0.43|
|             3|              0.06|       34|         0.33|
|             1|              0.02|       71|          0.7|
|             0|               0.0|       86|         0.85|
|             3|              0.06|       11|          0.1|
|             0|               0.0|       21|          0.2|
|             1|              0.02|       54|         0.53|
|             0|               0.0|       22|         0.21|
|             8|              0.16|        6|         0.05|
|             3|              0.06|       32|         0.31|
|             4|              0.08|       56|         0.55|
|             0|               0.0|       64|         0.63|
|             2|              0.04|     

In [ ]:
##Transform selected features into vectors

In [81]:
df_assembler=VectorAssembler(inputCols=['healsandboostsnorm','damageDealNormnorm','killsNormnorm','killPlacenorm','totalDistancenorm',
                                        'killStreaksnorm','longestKillnorm','walkDistancenorm','weaponsAcquirednorm'],outputCol="features")

df2=df_assembler.transform(df_norm)
model_df=df2.select(['features','winPlacePerc'])
model_df=model_df.withColumnRenamed("winPlacePerc","label")

In [172]:
model_df.show()

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(9,[3,4,7,8],[0.7...| 0.12|
|(9,[3,4,7,8],[0.4...| 0.96|
|[0.06,0.018743838...| 0.56|
|[0.02,0.0,0.0,0.7...| 0.24|
|(9,[3,4,7,8],[0.8...| 0.08|
|[0.06,0.020046007...|  1.0|
|[0.0,0.0519635228...| 0.16|
|[0.02,0.023599244...| 0.56|
|[0.0,0.0241948734...|  1.0|
|[0.16,0.068333059...| 0.88|
|[0.06,0.015404206...|  0.6|
|[0.08,0.0,0.0,0.5...| 0.56|
|[0.0,0.0092178770...|  0.4|
|[0.04,0.003415626...|  0.6|
|[0.0,0.0236197831...| 0.24|
|[0.12,0.058494906...| 0.44|
|[0.1,0.0679838974...| 0.96|
|[0.26,0.042967466...| 0.88|
|[0.08,0.096060630...|  1.0|
|[0.0,0.0410778836...| 0.44|
+--------------------+-----+
only showing top 20 rows



In [345]:
#Spilt the traing and testing set

In [86]:
training_df,testing_df=model_df.randomSplit([0.7,0.3])

In [87]:
training_df.count()

3102974

In [88]:
testing_df.count()

1328921

In [181]:
#Machine learning

In [89]:
#Random forest
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import RandomForestRegressor
rf=RandomForestRegressor()

In [119]:
from pyspark.ml import Pipeline
rf=RandomForestRegressor()
pipeline=Pipeline(stages=[rf])
model_rf=pipeline.fit(training_df)

In [120]:
start_time_rf=time.time()
predictions_rf=model_rf.transform(testing_df)
end_time_rf=time.time()
total_time_rf=end_time_rf-start_time_rf
print(total_time_rf)

0.19205093383789062


In [91]:
from pyspark.ml.evaluation import RegressionEvaluator
evaluator_r2=RegressionEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("r2")
evaluator_rmse=RegressionEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("rmse")

In [189]:
rf_r2=evaluator_r2.evaluate(predictions_rf)
print(rf_r2)

0.8416392257948355


In [191]:
rf_rmse=evaluator_rmse.evaluate(predictions_rf)
print(rf_rmse)

0.12218511109799837


In [192]:
predictions_rf.select('label','prediction').show()

+-----+-------------------+
|label|         prediction|
+-----+-------------------+
| 0.96| 0.7516882976399468|
|  1.0| 0.8500144531857918|
|  1.0| 0.7975617244314783|
|  0.6| 0.6730866272820697|
| 0.56|  0.584369640831339|
|  0.6| 0.6284058935854007|
| 0.44| 0.6403446265653774|
|  0.0|0.05617337490155224|
| 0.88| 0.6394291730631309|
|  0.8| 0.7856430936930188|
| 0.12|0.15654633297584974|
| 0.68| 0.7421312806728695|
| 0.12|0.09952942203286845|
| 0.08|0.11574264296603627|
| 0.48| 0.6598436245289085|
| 0.16|0.23152223037159209|
| 0.04|0.07253928510624309|
|  0.2|0.23583322363625708|
| 0.24| 0.2394166190337515|
|  0.6| 0.5682315956100107|
+-----+-------------------+
only showing top 20 rows



In [193]:
#Linear regression

In [115]:
from pyspark.ml.regression import LinearRegression
lr=LinearRegression()
from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression
pipeline=Pipeline(stages=[lr])
model_lr=pipeline.fit(training_df)

In [116]:
start_time_lr=time.time()
predictions_lr=model_lr.transform(testing_df)
end_time_lr=time.time()
total_time_lr=end_time_lr-start_time_lr
print(total_time_lr)

0.15981173515319824


In [198]:
lr_r2=evaluator_r2.evaluate(predictions_lr)
print(lr_r2)

0.8101519732061374


In [200]:
lr_rmse=evaluator_rmse.evaluate(predictions_lr)
print(lr_rmse)

0.13378195055122657


In [201]:
predictions_lr.select('label','prediction').show()

+-----+-------------------+
|label|         prediction|
+-----+-------------------+
| 0.96|  0.721494900653645|
|  1.0| 1.0412179682926368|
|  1.0| 0.9151380150970152|
|  0.6| 0.5451979372517126|
| 0.56| 0.5154738953603585|
|  0.6|0.49955316527680144|
| 0.44|0.45009640160008435|
|  0.0|0.05945708103491376|
| 0.88| 0.5544756111204812|
|  0.8| 0.6559685648106013|
| 0.12|0.18124330373031816|
| 0.68| 0.7571769981337068|
| 0.12| 0.1338954950657275|
| 0.08|0.13784648919876863|
| 0.48| 0.7586818035710097|
| 0.16|0.22264412044381254|
| 0.04|0.08608870269010682|
|  0.2|0.31965814162084744|
| 0.24|0.23114760025779102|
|  0.6| 0.5157254257006886|
+-----+-------------------+
only showing top 20 rows



In [202]:
#Decision tree

In [108]:
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml import Pipeline
dt=DecisionTreeRegressor()
pipeline=Pipeline(stages=[dt])
model_dt=pipeline.fit(training_df)

In [113]:
start_time_dt=time.time()
predictions_dt=model_dt.transform(testing_df)
end_time_dt=time.time()
total_time_dt=end_time_dt-start_time_dt
print(total_time_dt)

0.026291847229003906


In [207]:
dt_r2=evaluator_r2.evaluate(predictions_dt)
print(dt_r2)

0.8460742789290905


In [212]:
dt_rmse=evaluator_rmse.evaluate(predictions_dt)
print(dt_rmse)

0.12046200246783491


In [210]:
predictions_dt.select('label','prediction').show()

+-----+-------------------+
|label|         prediction|
+-----+-------------------+
| 0.96| 0.7716369376367671|
|  1.0| 0.9100684357380648|
|  1.0| 0.8403500925120844|
|  0.6|  0.700306524789523|
| 0.56| 0.6237132188008003|
|  0.6| 0.5753745571771057|
| 0.44| 0.6237132188008003|
|  0.0|0.03313655135362408|
| 0.88| 0.6237132188008003|
|  0.8| 0.8383453408102455|
| 0.12|0.16307433341327848|
| 0.68| 0.8383453408102455|
| 0.12|0.12459329629244982|
| 0.08|0.08001949396831547|
| 0.48| 0.5680900764503072|
| 0.16|0.20617157767833863|
| 0.04|0.08001949396831547|
|  0.2|0.16217701185829653|
| 0.24| 0.2833710977204408|
|  0.6| 0.5753745571771057|
+-----+-------------------+
only showing top 20 rows



In [94]:
#Tuning Linear regression

In [98]:
from pyspark.ml import Pipeline
from pyspark.ml.tuning import ParamGridBuilder,CrossValidator
from pyspark.ml.regression import LinearRegression
lr_1=LinearRegression(maxIter=10)
pipeline=Pipeline(stages=[lr_1])
model_lr_1=pipeline.fit(training_df)

In [99]:
start_time_lr_1=time.time()
predictions_lr_1=model_lr_1.transform(testing_df)
end_time_lr_1=time.time()
total_time_lr_1=end_time_lr_1-start_time_lr_1
print(total_time_lr_1)

lr_r2_1=evaluator_r2.evaluate(predictions_lr_1)
print(lr_r2_1)

lr_rmse_1=evaluator_rmse.evaluate(predictions_lr_1)
print(lr_rmse_1)

0.12936878204345703
0.8103265247942821
0.1336843098461082


In [100]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression
lr_2=LinearRegression(maxIter=15)
pipeline=Pipeline(stages=[lr_2])
model_lr_2=pipeline.fit(training_df)

In [101]:
start_time_lr_2=time.time()
predictions_lr_2=model_lr_2.transform(testing_df)
end_time_lr_2=time.time()
total_time_lr_2=end_time_lr_2-start_time_lr_2
print(total_time_lr_2)

lr_r2_2=evaluator_r2.evaluate(predictions_lr_2)
print(lr_r2_2)

lr_rmse_2=evaluator_rmse.evaluate(predictions_lr_2)
print(lr_rmse_2)

0.11773967742919922
0.8103366993170137
0.13368072423064695


In [ ]:
#Tuning decision tree

In [121]:
from pyspark.ml import Pipeline
dt_1=DecisionTreeRegressor(maxDepth=5)
pipeline=Pipeline(stages=[dt_1])
model_dt_1=pipeline.fit(training_df)

In [122]:
start_time_dt_1=time.time()
predictions_dt_1=model_dt_1.transform(testing_df)
end_time_dt_1=time.time()
total_time_dt_1=end_time_dt_1-start_time_dt_1
print(total_time_dt_1)

dt_r2_1=evaluator_r2.evaluate(predictions_dt_1)
print(dt_r2_1)

dt_rmse_1=evaluator_rmse.evaluate(predictions_dt_1)
print(dt_rmse_1)

0.06852865219116211
0.84617746487309
0.12030828955107345


In [123]:
from pyspark.ml import Pipeline
dt_2=DecisionTreeRegressor(maxDepth=7)
pipeline=Pipeline(stages=[dt_2])
model_dt_2=pipeline.fit(training_df)

In [124]:
start_time_dt_2=time.time()
predictions_dt_2=model_dt_2.transform(testing_df)
end_time_dt_2=time.time()
total_time_dt_2=end_time_dt_2-start_time_dt_2
print(total_time_dt_2)

dt_r2_2=evaluator_r2.evaluate(predictions_dt_2)
print(dt_r2_2)

dt_rmse_2=evaluator_rmse.evaluate(predictions_dt_2)
print(dt_rmse_2)

0.24312233924865723
0.8666530821002912
0.11201522643527245


In [ ]:
#Tuning random forest

In [94]:
from pyspark.ml import Pipeline
rf_1=RandomForestRegressor(maxDepth=3)
pipeline=Pipeline(stages=[rf_1])
model_rf_1=pipeline.fit(training_df)

In [95]:
start_time_rf_1=time.time()
predictions_rf_1=model_rf_1.transform(testing_df)
end_time_rf_1=time.time()
total_time_rf_1=end_time_rf_1-start_time_rf_1
print(total_time_rf_1)

rf_r2_1=evaluator_r2.evaluate(predictions_rf_1)
print(rf_r2_1)
rf_rmse_1=evaluator_rmse.evaluate(predictions_rf_1)
print(rf_rmse_1)

0.2435154914855957
0.8051480090486378
0.13549696309491824


In [96]:
from pyspark.ml import Pipeline
rf_2=RandomForestRegressor(maxDepth=4)
pipeline=Pipeline(stages=[rf_2])
model_rf_2=pipeline.fit(training_df)

In [97]:
start_time_rf_2=time.time()
predictions_rf_2=model_rf_2.transform(testing_df)
end_time_rf_2=time.time()
total_time_rf_2=end_time_rf_2-start_time_rf_2
print(total_time_rf_2)

rf_r2_2=evaluator_r2.evaluate(predictions_rf_2)
print(rf_r2_2)
rf_rmse_2=evaluator_rmse.evaluate(predictions_rf_2)
print(rf_rmse_2)

0.1118323802947998
0.8287961865793335
0.1270088122366782


In [ ]:
#Plot
#If you run the program, the plot will be shown on the popup window

In [1]:
import matplotlib
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
df=pd.read_csv('file:///home/cloudera/Downloads/train_V2.csv')

In [3]:
train=df

In [4]:
#kills count
labels0=['0','1','2','3','4','5','6','7','larger']
data0=[2529722,928079,472466,232441,124543,66577,37960,21816,33362]
plt.bar(range(len(data0)),data0)
plt.xticks(range(len(data0)),labels0)
plt.xlabel('kills')
plt.ylabel('count')
plt.title('Kill count')
matplotlib.pyplot.show()

In [5]:
plt.figure(figsize=(10,4))
counts=train['kills'].value_counts()
plt.bar(counts.index,counts.values,color='PowDerBlue')
plt.title('kills',fontsize=15)
plt.xlabel('kills')
plt.ylabel('count')
plt.grid()
plt.show()

In [6]:
train['playersJoined']=train.groupby('matchId')['matchId'].transform('count')

In [7]:
#playersJoined
plt.figure(figsize=(20,10))
counts=train['playersJoined'].value_counts()
plt.bar(counts.index,counts.values,color='PowDerBlue')
plt.title('playersJoined',fontsize=15)
plt.xlabel('playersJoined')
plt.ylabel('count')
plt.grid()
plt.show()

In [8]:
select_playersJoined=train[train['playersJoined']>=75]

In [9]:
#playersJoined
plt.figure(figsize=(20,10))
counts=train[train['playersJoined']>=75]['playersJoined'].value_counts()
plt.bar(counts.index,counts.values,color='PowDerBlue')
plt.title('playersJoined>=75',fontsize=15)
plt.xlabel('playersJoined')
plt.ylabel('count')
plt.grid(True)
plt.show()

In [10]:
train['headshot_rate']=train['headshotKills']/train['kills']
train['headshot_rate']=train['headshot_rate'].fillna(0)
train['headshot_rate'].tail()

4446961    0.0
4446962    0.0
4446963    0.0
4446964    0.5
4446965    0.0
Name: headshot_rate, dtype: float64

In [11]:
#headshot rate
plt.figure(figsize=(12,4))
plt.hist(train['headshot_rate'],bins=10,color='PowDerBlue')
plt.title('headshot_rate',fontsize=15)
plt.xlabel('headshot_rate')
plt.ylabel('count')
plt.grid(True)
plt.show()

In [12]:
#longest kill
plt.figure(figsize=(12,4))
plt.hist(train['longestKill'],bins=10,color='PowDerBlue')
plt.title('longestKill',fontsize=15)
plt.xlabel('longestKil')
plt.grid(True)
plt.show()

In [13]:
#road kills
labels=['0','1','2','3','4','5','6','7','8','9','10','11','14','18']
data=[4433970,11247,1298,297,72,36,20,11,7,2,2,2,1,1]
plt.figure(figsize=(12,4))
plt.bar(range(len(data)),data,color='PowDerBlue')

plt.xticks(range(len(data)),labels)
plt.xlabel('roadKills')
plt.ylabel('count')
plt.title('kills on drive',fontsize=15)
plt.grid(True)
plt.show()

In [14]:
#rideDistance
data = train[train['rideDistance'] < train['rideDistance'].quantile(0.99)]
plt.figure(figsize=(12,4))
plt.title("The ride Distances",fontsize=15)
plt.hist(data['rideDistance'],bins=10,color='PowDerBlue')  #distplot直方图
plt.xlabel('rideDistance')
plt.grid(True)
plt.show()

In [15]:
#heals
data = train[train['heals'] < train['heals'].quantile(0.99)]
plt.figure(figsize=(12,4))
plt.hist(data['heals'],bins=10,color='PowDerBlue')  #distplot直方图
plt.title("The therapeutic drugs",fontsize=15)
plt.xlabel('therapeuticDrugs')
plt.grid(True)
plt.show()

In [16]:
#swimDisatance
data = train[train['swimDistance'] < train['swimDistance'].quantile(0.99)]
plt.figure(figsize=(12,4))
#plt.title("The ride Distances")
plt.hist(data['swimDistance'],bins=10,color='PowDerBlue')
plt.title("The swim Distances",fontsize=15)
plt.xlabel('swimDistance')
plt.grid(True)
plt.show()

In [17]:
#weaponsAcquired
labels2=['0','1','2','3','4','5','6','7','8','9','10','larger']
data2=[226578,580951,768836,768347,689622,540721,369549,226776,128698,68479,35642,42767]
plt.figure(figsize=(12,4))
plt.bar(range(len(data2)),data2,color='PowDerBlue')
plt.xticks(range(len(data2)),labels2)
plt.ylabel('count')
plt.title("The weapons acquired",fontsize=15)
plt.xlabel('weaponsAcquired')
plt.grid(True)


plt.show()

In [18]:
#walkDistance
data=train.copy()
#data = train[train['walkDistance'] < train['walkDistance'].quantile(0.99)]
plt.figure(figsize=(12,4))
plt.hist(data['walkDistance'],bins=10,color='PowDerBlue')  #distplot直方图

plt.xlabel('walkingDistance')
plt.title('walking distance',fontsize=15)
plt.grid(True)
plt.show()

In [20]:
#playersJoined boxplot
playersJioned=train['playersJoined']
plt.figure(figsize=(20,10))
plt.boxplot(playersJioned)
plt.show()

In [21]:
def box_outlier(data):
    df = data.copy(deep=True)
    out_index = []
    for col in df.columns:            
        Q1 = df[col].quantile(q=0.25)      
        Q3 = df[col].quantile(q=0.75)       
        low_whisker = Q1 - 1.5 * (Q3 - Q1)  
        up_whisker = Q3 + 1.5 * (Q3 - Q1)   
        rule = (df[col] > up_whisker) | (df[col] < low_whisker)  
        out = df[col].index[rule]
        out_index += out.tolist()  
    df.drop(out_index, inplace=True)
    return df


In [22]:
players=train[['playersJoined','walkDistance']]
players_dele=box_outlier(players)
playersJioned=players_dele['playersJoined']

In [23]:
#the playersJoined boxplot after deleting outliers
plt.figure(figsize=(20,10))
plt.boxplot(playersJioned)
plt.show()

In [25]:
fig_q2 = plt.figure(figsize = (22,12))


ax1 = fig_q2.add_subplot(2,3,1) 
ax2 = fig_q2.add_subplot(2,3,2) 
ax3 = fig_q2.add_subplot(2,3,3) 
ax4 = fig_q2.add_subplot(2,3,4) 
ax5 = fig_q2.add_subplot(2,3,5) 
ax6 = fig_q2.add_subplot(2,3,6)




ax1.bar(range(len(data0)),data0,color='PowDerBlue')
ax1.set_xticks(range(len(data0)),labels0)
ax1.set_xlabel('kills')
#ax1.set_ylabel('count')
ax1.set_title('Kill count',fontsize=15)
ax1.grid(True)


ax2.set_title("Damage Dealt",fontsize=15)
ax2.hist(train['damageDealt'],bins=10,color='PowDerBlue')
ax2.set_xlabel('damageDealt')
#ax2.set_ylabel('count')
ax2.grid(True)


data_kill0 = data[data['kills']==0]
ax3.set_title("Damage Dealt by 0 killers",fontsize=15)
ax3.hist(data_kill0['damageDealt'],bins=10,color='PowDerBlue')
ax3.set_xlabel('danageDealt')
ax3.grid(True)


ax4.set_title("The walking Distances",fontsize=15)
ax4.hist(data['walkDistance'],bins=10,color='PowDerBlue')
ax4.set_xlabel('walkDistance')
ax4.grid(True)

winner_data=train[train['winPlacePerc'] == 1].copy()
winner_data = winner_data[winner_data['walkDistance'] < winner_data['walkDistance'].quantile(0.99)]
ax5.set_title("The winner's walking distances",fontsize=15)
ax5.hist(winner_data['walkDistance'],bins=10,color='PowDerBlue')  #distplot直方图
ax5.set_xlabel('walkDistance')
ax5.grid(True)

ax6.set_title("weaponsAcquired count",fontsize=15)
ax6.bar(range(len(data2)),data2,color='PowDerBlue')
ax6.set_xticks(range(len(data2)),labels2)
ax6.set_xlabel('weaponsAcquired')
#ax6.set_ylabel('count')
ax6.grid(True)


plt.show()

In [26]:
#R2 of models
labels5=['Linear regression','Decision tree','Random forest']
data5=[81.03,86.67,84.16]
plt.figure(figsize=(10,10))
plt.bar(range(len(data5)),data5,color='PowDerBlue',width=0.3,align='center')
plt.xticks(range(len(data5)),labels5)
plt.xlabel('model',fontsize=15)
plt.ylabel('R2',fontsize=15)
plt.title('R2 of each model',fontsize=15)
plt.grid(True)
for x,y in zip(range(len(data5)),data5):
    plt.text(x,y,y,fontsize=13,horizontalalignment='center')
plt.show()

In [27]:
#RMSE of all models
labels5=['Linear regression','Decision tree','Random forest']
data5=[0.13368,0.12031,0.122185]
plt.figure(figsize=(10,10))
plt.bar(range(len(data5)),data5,color='PowDerBlue',width=0.3,align='center')
plt.xticks(range(len(data5)),labels5)
plt.xlabel('model',fontsize=15)
plt.ylabel('RMSE',fontsize=15)
plt.title('RMSE of each model',fontsize=15)
plt.grid(True)
for x,y in zip(range(len(data5)),data5):
    plt.text(x,y,y,fontsize=13,horizontalalignment='center')
plt.show()


In [28]:
#Running time of all models
labels5=['Linear regression','Decision tree','Random forest']
data5=[0.1598,0.06853 ,0.19205]
plt.figure(figsize=(10,10))
plt.bar(range(len(data5)),data5,color='PowDerBlue',width=0.3,align='center')
plt.xticks(range(len(data5)),labels5)
plt.xlabel('model',fontsize=15)
plt.ylabel('time',fontsize=15)
plt.title('Running time of each model',fontsize=15)
plt.grid(True)
for x,y in zip(range(len(data5)),data5):
    plt.text(x,y,y,fontsize=13,horizontalalignment='center')
plt.show()


In [1]:
pip install statsmodels

Note: you may need to restart the kernel to use updated packages.


In [2]:
import statsmodels.api

/Users/yaoyutong/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
